## This is an notebook includes code for
- Calculating Accuracy of scores
- Measuring improvement of Vadar and XLM-T with ensemble methods.
- Measuring improvement of Final method with ensemble methods

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
emotion_scores =[]
sentiment_scores =[]
for i in range(12):
    file_path = f"/content/drive/MyDrive/tiwtter_sentiment_big_file/emotion_scores_{i}.txt"
    with open(file_path,"r") as file:
        emotions = eval(file.read())
    #print(len(emotions))
    emotion_scores.extend(emotions)

print(len(emotion_scores))
for i in range(12):
    file_path = f"/content/drive/MyDrive/tiwtter_sentiment_big_file/sentiment_scores_{i}.txt"
    with open(file_path,"r") as file:
        sentiments = eval(file.read())
    #print(len(emotions))
    sentiment_scores.extend(sentiments)

print(len(sentiment_scores))
import numpy as np
import pandas as pd

import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/tiwtter_sentiment_big_file/submission_train.csv')
samuels_pred_df = pd.read_csv('/content/drive/MyDrive/tiwtter_sentiment_big_file/training_no_duplicates.csv')
num_rows =1127644
# Split the DataFrame into two parts
first_part = train_df.iloc[:num_rows]
second_part = train_df.iloc[num_rows:]

print(len(first_part))
print(len(second_part))
print(len(first_part)+len(second_part))
# Concatenate the second part first, followed by the first part
train_df = pd.concat([second_part, first_part]).reset_index(drop=True)
fe = emotion_scores[:1127644]
se = emotion_scores[1127644:]
emotion_score_swapped = se + fe
print(len(emotion_score_swapped))

fs = sentiment_scores[:1127644]
ss = sentiment_scores[1127644:]
sentiment_score_swapped = ss + fs
train_df.loc[:, 'emotion_scores'] = emotion_score_swapped

train_df.loc[:, 'sentiment_scores2'] = sentiment_score_swapped

train_df.loc[:, 'sentiment_scores'] = samuels_pred_df['predictions']
train_df.loc[:, 'sentiment_scores'] = train_df.loc[:, 'sentiment_scores'].replace(0,-1)
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].replace(0, np.nan)
train_df.loc[:,'spam_score'] = train_df['spam_score'].replace(0,np.nan)
train_df.loc[:,'fake_lables'] = train_df['fake_lables'].replace(0,np.nan)
train_df.loc[:,'spam_score']= train_df['spam_score'].replace(1,-1)
def modify_value(x):
    if x == 0.0 or np.isnan(x):
        return np.nan
    elif x > 0.0:
        return 1
    else:
        return -1
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].apply(modify_value)


2270482
2270482
1127644
1142838
2270482
2270482


In [ ]:
old_vadar_score_df = pd.read_csv('/content/drive/MyDrive/tiwtter_sentiment_big_file/full_df_big_deprecated.csv')
old_vadar_score_df.loc[:, 'vadar'] = old_vadar_score_df['vadar'].apply(modify_value)
vader_scores = old_vadar_score_df['vadar']
first_part = vader_scores.iloc[:num_rows]
second_part = vader_scores.iloc[num_rows:]

vader_scores_swapped = pd.concat([second_part, first_part]).reset_index(drop=True)
train_df.loc[:, 'vadar'] = vader_scores_swapped

In [ ]:
from sklearn.metrics import accuracy_score,precision_score, recall_score
from sklearn.metrics import confusion_matrix
emoji_scores = train_df['emoji_scores']
emoji_scores_cal_df = pd.DataFrame({'label': train_df['label'], 'emoji_scores': emoji_scores})
# Filter out rows where the 'vader_scores' is NaN
filtered_df = emoji_scores_cal_df.dropna(subset=['emoji_scores'])
print(f"length of non-np.nan elements {len(filtered_df)/2270482}")
print(f"length of pos elements {len(filtered_df[filtered_df['emoji_scores']==1])/len(filtered_df)}")
print(f"length of neg elements {len(filtered_df[filtered_df['emoji_scores']==-1])/len(filtered_df)}")
from sklearn.metrics import accuracy_score,precision_score, recall_score
accuracy_sam = accuracy_score(filtered_df['label'], filtered_df['emoji_scores'])
precision_sam = precision_score(filtered_df['label'], filtered_df['emoji_scores'])
recall_sam = recall_score(filtered_df['label'], filtered_df['emoji_scores'])
precision_sam_neg = precision_score(filtered_df['label'], filtered_df['emoji_scores'],pos_label =-1)
recall_sam_neg = recall_score(filtered_df['label'], filtered_df['emoji_scores'],pos_label =-1)

cm = confusion_matrix(filtered_df['label'], filtered_df['emoji_scores'])
print(cm)
print(f"accuracy: {accuracy_sam:.2f}")
print(f"precision_pos: {precision_sam:.2f}")
print(f"recall_pos: {recall_sam:.2f}")
print(f"precision_neg: {precision_sam_neg:.2f}")
print(f"recall_neg: {recall_sam_neg:.2f}")

length of non-np.nan elements 0.2273935666523672
length of pos elements 0.24731305673328904
length of neg elements 0.7526869432667109
[[377402  27520]
 [ 11205 100166]]
accuracy: 0.92
precision_pos: 0.78
recall_pos: 0.90
precision_neg: 0.97
recall_neg: 0.93


In [ ]:
# Define the function to modify the column values
def modify_value(x):
    if x ==0 or np.isnan(x):
        return np.nan
    else:
        return x

# Apply the function to the specified column
fake_lables_scores = train_df['fake_lables'].apply(modify_value)
fake_lables_scores_cal_df = pd.DataFrame({'label': train_df['label'], 'fake_lables': fake_lables_scores})
# Filter out rows where the 'vader_scores' is NaN
filtered_df = fake_lables_scores_cal_df.dropna(subset=['fake_lables'])
print(f"length of non-np.nan elements {len(filtered_df)/2270482}")
print(f"length of pos elements {len(filtered_df[filtered_df['fake_lables']==1])}")
print(f"length of neg elements {len(filtered_df[filtered_df['fake_lables']==-1])}")
print(f"part of pos elements {len(filtered_df[filtered_df['fake_lables']==1])/len(filtered_df)}")
print(f"part of neg elements {len(filtered_df[filtered_df['fake_lables']==-1])/len(filtered_df)}")
from sklearn.metrics import accuracy_score,precision_score, recall_score
accuracy_sam = accuracy_score(filtered_df['label'], filtered_df['fake_lables'])
precision_sam = precision_score(filtered_df['label'], filtered_df['fake_lables'])
recall_sam = recall_score(filtered_df['label'], filtered_df['fake_lables'])
precision_sam_neg = precision_score(filtered_df['label'], filtered_df['fake_lables'],pos_label =-1)
recall_sam_neg = recall_score(filtered_df['label'], filtered_df['fake_lables'],pos_label =-1)


cm = confusion_matrix(filtered_df['label'], filtered_df['fake_lables'])
print(cm)
print(f"accuracy: {accuracy_sam:.2f}")
print(f"precision_pos: {precision_sam:.2f}")
print(f"recall_pos: {recall_sam:.2f}")
print(f"precision_neg: {precision_sam_neg:.2f}")
print(f"recall_neg: {recall_sam_neg:.2f}")


length of non-np.nan elements 0.19498282743487946
length of pos elements 63780
length of neg elements 378925
part of pos elements 0.1440688494595724
part of neg elements 0.8559311505404276
[[371236   7796]
 [  7689  55984]]
accuracy: 0.97
precision_pos: 0.88
recall_pos: 0.88
precision_neg: 0.98
recall_neg: 0.98


In [ ]:

# Apply the function to the specified column
spam_scores = train_df['spam_score']
spam_scores_cal_df = pd.DataFrame({'label': train_df['label'], 'spam_scores': spam_scores})
# Filter out rows where the 'vader_scores' is NaN
filtered_df = spam_scores_cal_df.dropna(subset=['spam_scores'])
print(f"length of non-np.nan elements {len(filtered_df)/2270482}")
print(f"length of pos elements {len(filtered_df[filtered_df['spam_scores']==1])}")
print(f"length of neg elements {len(filtered_df[filtered_df['spam_scores']==-1])}")
from sklearn.metrics import accuracy_score,precision_score, recall_score
accuracy_sam = accuracy_score(filtered_df['label'], filtered_df['spam_scores'])
precision_sam_pos = precision_score(filtered_df['label'], filtered_df['spam_scores'])
precision_sam_neg = precision_score(filtered_df['label'], filtered_df['spam_scores'],pos_label =-1)
recall_sam_neg = recall_score(filtered_df['label'], filtered_df['spam_scores'],pos_label =-1)

cm = confusion_matrix(filtered_df['label'], filtered_df['spam_scores'])
print(cm)
print(f"accuracy: {accuracy_sam:.4f}")
print(precision_sam_pos)
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")


length of non-np.nan elements 0.050906371422455673
length of pos elements 0
length of neg elements 115582


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[115413      0]
 [   169      0]]
accuracy: 0.9985
0.0
precision_neg: 0.9985
recall_neg: 1.0000


In [ ]:
print(f"true pos: {precision_sam_pos:.4f}")

true pos: 0.0000


## Measuring Improvement of using ensemble method

This is XLM-T's prediction accuracy

In [ ]:
from sklearn.metrics import accuracy_score,precision_score, recall_score
from sklearn.metrics import confusion_matrix

In [ ]:

accuracy_sam = accuracy_score(train_df['label'], train_df['sentiment_scores2'])
precision_sam = precision_score(train_df['label'], train_df['sentiment_scores2'])
recall_sam = recall_score(train_df['label'], train_df['sentiment_scores2'])
precision_sam_neg = precision_score(train_df['label'], train_df['sentiment_scores2'],pos_label =-1)
recall_sam_neg = recall_score(train_df['label'], train_df['sentiment_scores2'],pos_label =-1)
cm = confusion_matrix(train_df['label'], train_df['sentiment_scores2'])
print(cm)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

[[689734 453104]
 [228664 898980]]
accuracy: 0.6997
precision_pos: 0.6649
recall_pos: 0.7972
precision_neg: 0.7510
recall_neg: 0.6035


In [ ]:

vader_scores = train_df['vadar']
vadar_score_cal_df = pd.DataFrame({'label': train_df['label'], 'vader_scores': vader_scores})
# Filter out rows where the 'vader_scores' is NaN
filtered_df = vadar_score_cal_df.dropna(subset=['vader_scores'])
print(f"length of non-np.nan elements {len(filtered_df)/2270482}")
print(f"length of pos elements {len(filtered_df[filtered_df['vader_scores']==1])/len(filtered_df)}")
print(f"length of neg elements {len(filtered_df[filtered_df['vader_scores']==-1])/len(filtered_df)}")
# Compute accuracy only on non-NaN values
accuracy = accuracy_score(filtered_df['label'], filtered_df['vader_scores'])
from sklearn.metrics import accuracy_score,precision_score, recall_score
accuracy_sam = accuracy_score(filtered_df['label'], filtered_df['vader_scores'])
precision_sam = precision_score(filtered_df['label'], filtered_df['vader_scores'])
recall_sam = recall_score(filtered_df['label'], filtered_df['vader_scores'])
precision_sam_neg = precision_score(filtered_df['label'], filtered_df['vader_scores'],pos_label =-1)
recall_sam_neg = recall_score(filtered_df['label'], filtered_df['vader_scores'],pos_label =-1)

cm = confusion_matrix(filtered_df['label'], filtered_df['vader_scores'])
print(cm)
print(f"accuracy: {accuracy_sam:.2f}")
print(f"precision_pos: {precision_sam:.2f}")
print(f"recall_pos: {recall_sam:.2f}")
print(f"precision_neg: {precision_sam_neg:.2f}")
print(f"recall_neg: {recall_sam_neg:.2f}")

length of non-np.nan elements 0.7463719157430009
length of pos elements 0.677930915648545
length of neg elements 0.322069084351455
[[414129 414532]
 [131657 734306]]
accuracy: 0.68
precision_pos: 0.64
recall_pos: 0.85
precision_neg: 0.76
recall_neg: 0.50


This is Embedding approach's prediction accuracy

In [ ]:
from sklearn.metrics import accuracy_score,precision_score, recall_score
from sklearn.metrics import confusion_matrix
accuracy_sam = accuracy_score(train_df['label'], train_df['sentiment_scores'])
precision_sam = precision_score(train_df['label'], train_df['sentiment_scores'])
recall_sam = recall_score(train_df['label'], train_df['sentiment_scores'])
precision_sam_neg = precision_score(train_df['label'], train_df['sentiment_scores'],pos_label =-1)
recall_sam_neg = recall_score(train_df['label'], train_df['sentiment_scores'],pos_label =-1)
cm = confusion_matrix(train_df['label'], train_df['sentiment_scores'])
print(cm)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

[[ 994855  147983]
 [ 121312 1006332]]
accuracy: 0.8814
precision_pos: 0.8718
recall_pos: 0.8924
precision_neg: 0.8913
recall_neg: 0.8705


This is prediction accuracy of emotion scores

In [ ]:
from sklearn.metrics import accuracy_score,precision_score, recall_score
accuracy_sam = accuracy_score(train_df['label'], train_df['emotion_scores'])
precision_sam = precision_score(train_df['label'], train_df['emotion_scores'])
recall_sam = recall_score(train_df['label'], train_df['emotion_scores'])
precision_sam_neg = precision_score(train_df['label'], train_df['emotion_scores'],pos_label =-1)
recall_sam_neg = recall_score(train_df['label'], train_df['emotion_scores'],pos_label =-1)

cm = confusion_matrix(train_df['label'], train_df['emotion_scores'])
print(cm)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

[[622648 520190]
 [181361 946283]]
accuracy: 0.6910
precision_pos: 0.6453
recall_pos: 0.8392
precision_neg: 0.7744
recall_neg: 0.5448


## Measuring improvement of baseline models when we add those scores

In [ ]:

## Nulling out the emotion_scores
train_df.loc[train_df['emoji_scores'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['emoji_scores'] == -1, 'vadar'] = np.nan
train_df.loc[train_df['fake_lables'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['fake_lables'] == -1, 'vadar'] = np.nan
train_df.loc[train_df['spam_score'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['spam_score'] == -1, 'vadar'] = np.nan

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores2']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores for the positive class

In [ ]:
accuracy_sam = accuracy_score(y_test, y_pred)
precision_sam = precision_score(y_test, y_pred)
recall_sam = recall_score(y_test, y_pred)
precision_sam_neg = precision_score(y_test, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_test, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8059
precision_pos: 0.7889
recall_pos: 0.8322
precision_neg: 0.8247
recall_neg: 0.7799


In [ ]:
train_df.head()

,line,label,emoji_scores,fake_lables,spam_score,vadar,emotion_scores,sentiment_scores2,sentiment_scores
0,vinco tresorpack <number> difficulty <number> ...,-1,-1.0,-1.0,NaN,NaN,NaN,1,-1
1,glad i dot have taks tomorrow ! ! thankful sta...,-1,NaN,NaN,NaN,1.0,1.0,1,1
2,<number> <number> vs celtics in the regular se...,-1,NaN,NaN,NaN,-1.0,-1.0,-1,-1
3,<user> i could actually kill that girl i am so...,-1,NaN,NaN,NaN,-1.0,-1.0,-1,-1
4,<user> <user> <user> i find that very hard to ...,-1,NaN,NaN,NaN,-1.0,-1.0,-1,-1


Improvement for vadar

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'vadar','emotion_scores']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores for the positive class

In [ ]:
accuracy_sam = accuracy_score(y_test, y_pred)
precision_sam = precision_score(y_test, y_pred)
recall_sam = recall_score(y_test, y_pred)
precision_sam_neg = precision_score(y_test, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_test, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8053
precision_pos: 0.7877
recall_pos: 0.8325
precision_neg: 0.8247
recall_neg: 0.7784


Improvement on our final submission

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'sentiment_scores','emotion_scores']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores for the positive class

In [ ]:
accuracy_sam = accuracy_score(y_test, y_pred)
precision_sam = precision_score(y_test, y_pred)
recall_sam = recall_score(y_test, y_pred)
precision_sam_neg = precision_score(y_test, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_test, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8815
precision_pos: 0.8722
recall_pos: 0.8922
precision_neg: 0.8910
recall_neg: 0.8709


## Using Vadar and sentiment scores from xlm-t on top of the final prediction

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'sentiment_scores','emotion_scores','vadar','sentiment_scores2']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores for the positive class

In [ ]:
accuracy_sam = accuracy_score(y_test, y_pred)
precision_sam = precision_score(y_test, y_pred)
recall_sam = recall_score(y_test, y_pred)
precision_sam_neg = precision_score(y_test, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_test, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8815
precision_pos: 0.8722
recall_pos: 0.8922
precision_neg: 0.8910
recall_neg: 0.8709
